In [1]:
import pandas as pd

In [3]:
df_merge= pd.read_csv('../../Data/날씨+일별주차이용.csv')

In [7]:
pd.read_csv('../../Data/한강2403-2409.csv')
df = pd.read_csv('../../Data/서울시 한강공원 월별 이용객 현황.csv',encoding='euc-kr')


In [20]:

print(df.columns)
df = df.loc[:,['공원 코드','공원명','일반이용자(아침)','일반이용자(낮)', '일반이용자(저녁)','현황 일시']]

Index(['공원 코드', '공원명', '등록 일시', '일반이용자(아침)', '일반이용자(낮)', '일반이용자(저녁)', '현황 일시'], dtype='object')


In [24]:
df['현황 일시']= pd.to_datetime(df['현황 일시'])

In [ ]:
df_merge

0         5
1        11
2         1
3         8
4         8
         ..
45272     8
45273     1
45274     8
45275     1
45276     8
Name: code, Length: 45277, dtype: int64

In [ ]:
# PLT-001: 강서   
# PLT-002: 난지   
# PLT-003: 망원   
# PLT-004: 이촌   
# PLT-005: 뚝섬       
# PLT-006: 광나루     
# PLT-007: 양화   
# PLT-008: 여의도     
# PLT-009: 반포   
# PLT-0010: 잠원  
# PLT-0011: 잠실
                             
# Hzone011: 강서  
# Hzone009: 난지  
# Hzone010: 망원  
# Hzone006: 이촌  
# Hzone003: 뚝섬  
# Hzone001 광나루     
# Hzone008: 양화  
# Hzone007: 여의도    
# Hzone005: 반포  
# Hzone004: 잠원  
# Hzone002: 잠실  

In [36]:
# 주차장일별['code'] 
codelist = []
for code in df['공원 코드']:
    codelist.append(code.replace('Hzone',""))

In [38]:
df['code'] = codelist

In [47]:
df['code']= pd.to_numeric(df['code'])

In [54]:
# df.iloc[:,]
df_merge['날짜']= pd.to_datetime(df_merge['날짜'])

In [56]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45277 entries, 0 to 45276
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   지구별            45277 non-null  object        
 1   주차장명           45277 non-null  object        
 2   주차대수           45277 non-null  int64         
 3   이용시간           45277 non-null  float64       
 4   날짜             45277 non-null  datetime64[ns]
 5   code           45277 non-null  int64         
 6   년도             45277 non-null  int64         
 7   월              45277 non-null  int64         
 8   일              45277 non-null  int64         
 9   휴일여부           45277 non-null  object        
 10  지점명            45277 non-null  object        
 11  일시             45277 non-null  object        
 12  평균기온(°C)       45277 non-null  float64       
 13  최저기온(°C)       45277 non-null  float64       
 14  최저기온 시각(hhmi)  45277 non-null  float64       
 15  최고기온(°C)       4527

In [58]:
df.rename(columns={'현황 일시' : '날짜'}, inplace= True)

In [75]:
df

,공원 코드,공원명,일반이용자(아침),일반이용자(낮),일반이용자(저녁),날짜,code
0,Hzone001,광나루한강공원,2840.0,33780,25450,2024-02-29,1
1,Hzone006,이촌한강공원,5145.0,47990,37857,2024-02-29,6
2,Hzone003,뚝섬한강공원,13100.0,167450,187420,2024-02-29,3
3,Hzone002,잠실한강공원,9820.0,41340,49010,2024-02-29,2
4,Hzone008,양화한강공원,11150.0,33940,32050,2024-02-29,8
...,...,...,...,...,...,...,...
810,Hzone002,잠실한강공원,12700.0,53510,45450,2018-01-31,2
811,Hzone007,여의도한강공원,17920.0,100690,113010,2018-01-31,7
812,Hzone008,양화한강공원,3860.0,37975,18209,2018-01-31,8
813,Hzone003,뚝섬한강공원,44450.0,449660,129590,2018-01-31,3


In [81]:
df= df.iloc[:,2:]